In [4]:
#pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 1.2 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


# Stock capital gain calculation
## First in First out law (FIFO)

### - Import libraries

In [1]:
import pandas as pd
import numpy as np

### Extract required data 

In [2]:
# Read excel file
df = pd.read_excel("tax_2023.xlsx", sheet_name="US_2023")

# Clean the name columns
df['Code'] = df['SYMBOL']
df['Date'] = df['DATE (US)']
df['Type'] = df['SIDE']
df['Quantity'] = df['UNIT']
df['Total Value ($)'] = df['TOTAL']

# Set a filter for neccessary data
col_list = ['Code', 'Date', 'Type', 'Quantity', 'Total Value ($)']

# Filter the dataframe to keep the data required
df = df[col_list]

# Drop NaN rows
df.dropna(inplace=True)

# Drop duplicate name rows
filter = df['Type'] != 'Type'
df = df[filter]

# Change data type from string to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True) # Date : String to datetime format

# Sort data for Code names and event Date
df.sort_values(['Code', 'Date'], ascending=True, inplace=True) # Sorting multiple names [Code, Date]

# Reset the index numbers
df.reset_index(drop=True, inplace=True) # Reset index from 0
df['Unit Value ($)'] = df['Total Value ($)'] / df['Quantity']
df

,Code,Date,Type,Quantity,Total Value ($),Unit Value ($)
0,AAPL,2021-11-30,B,4,917.974080,229.493520
1,AAPL,2021-12-21,B,6,1433.048400,238.841400
2,AAPL,2021-12-21,B,6,1432.285550,238.714258
3,AAPL,2022-01-07,B,10,2397.448480,239.744848
4,AAPL,2022-07-11,S,-16,-3423.040668,213.940042
...,...,...,...,...,...,...
180,TSLA,2022-10-03,B,1,378.001008,378.001008
181,TSLA,2022-10-03,B,2,757.820514,378.910257
182,TSLA,2022-10-10,B,2,722.034880,361.017440
183,TSLA,2022-10-18,S,-3,-1051.358490,350.452830


### Capital Gain Calculator: FIFO

In [3]:
import logging
from collections import deque
import math

class Trans:
    datetime=None
    amount=None
    price=None
 
    def __init__(self, datetime, amount, price):
        self.datetime=datetime
        self.amount=amount
        self.price=price
     
    def getInfo(self):
        return(str(self.datetime)+"; "+
                str(self.amount)+"; "+
                str(self.price))+"; "
 
def balanceFifo(all_trans):
 
    qTransactions = deque() 
    trans_result = list()
 
    for t in all_trans:
        #Add first element to the queue
        if len(qTransactions)==0:
            #logging.debug('Added the first element: %s',t.getInfo())
            qTransactions.append(t)
            continue
 
        while (t.amount!=0 and len(qTransactions)>0):
            #investigate the first element from the queue
            tq=qTransactions.popleft()
            #the same type of transaction: both sell or both buy
            if tq.amount*t.amount>0:
                #return the first element back to the same place
                qTransactions.appendleft(tq)
                #add the new element to the list
                qTransactions.append(t)
                #logging.debug('Added: %s',t.getInfo())
                break
             
            #contrary transactions: (sell and buy) or (buy and sell) 
            if tq.amount*t.amount<0:
                #logging.debug('Transaction : %s',t.getInfo())
                #logging.debug('... try to balance with: %s',tq.getInfo())
 
                #The element in the queue have more units and takes in the current transaction
                if abs(tq.amount)>abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                            math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount of the element in the queue
                    tq.amount=tq.amount+t.amount
                    #return the element back to the same place
                    qTransactions.appendleft(tq)
                    #logging.debug('Removed transaction: %s',t.getInfo())
                    #the transaction has been balanced, take a new transaction
                    break
                 
                #The element from the queue and transaction have the same amount of units
                if abs(tq.amount)==abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                                math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount in the transaction 
                    t.amount=0
                    #logging.debug('Balanced, removed transaction: %s',t.getInfo())
                    #logging.debug('Balanced, removed from the queue: %s',tq.getInfo())
                    #the transaction has been balanced, take a new transaction
                    continue
                    
                #The transaction has more units
                if abs(tq.amount)<abs(t.amount):
                    #update the units in transaction, (remove element from the queue)
                    t.amount=t.amount+tq.amount
                    result = insertTransaction(tq.datetime,t.datetime,tq.amount,tq.price,t.price)
                    trans_result.append(result)
                    #logging.debug('Removed from queue: %s',tq.getInfo())
                     
                    #the transaction has not been balanced, 
                    #take a new element from the queue (t.amount>0)
                    continue
                 
        #We have unbalanced transaction but the queue is empty            
        if (t.amount!=0 and len(qTransactions)==0):
            #Add unbalanced transaction to the queue
            #The queue changes polarisation
            qTransactions.append(t)
            #logging.debug('Left element: %s',t.getInfo())
     
     
    #If something remained in the queue, treat it as open or part-open transactions
    while (len(qTransactions)>0):
        tq=qTransactions.popleft()
        #logging.debug('Remained on list transaction: %s',tq.getInfo())
        
    return trans_result
 
def insertTransaction(dateStart,dateEnd,amount,priceStart,priceEnd):
    #print("Bought={}, sold={},  amount={}, buy price={}, sell_price={}, gain={}".\
    #        format(dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)))
    result = [dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)]
    return result

### Calculate Capital Gain for Taxation using FIFO calculator

In [4]:
# Extract code names, numbers and counts
code = df['Code'].value_counts()
print("size: ", code.size, "index: ", code.index, "value: ", code.values)

size:  11 index:  Index(['SQQQ', 'TQQQ', 'SOXL', 'AAPL', 'QYLD', 'TSLA', 'CVX', 'OXY', 'DBA',
       'JNJ', 'SNOW'],
      dtype='object', name='Code') value:  [64 55 31  7  6  6  5  5  2  2  2]


In [5]:
# For loop for each Code strings 

start_date = pd.to_datetime("2022-06-30")
end_date = pd.to_datetime("2023-07-01")
sum_df = []

for j in range(0, code.size):
    # Seperate dataframe by Code name
    mask_code = df['Code'] == code.index[j]
    df_mask = df[mask_code]
    df_mask.reset_index(drop=True, inplace=True)
    print("All Transactions for", code.index[j])
    print(df_mask)
    
    # Capital gain caluculation FIFO
    trans_list=list()
    for i in range(0, df_mask.shape[0]):
        #trans = Trans(df_mask['Date'][i].date(), df_mask['Quantity'][i], df_mask['Unit Value ($)'][i])
        trans = Trans(pd.to_datetime(df_mask['Date'][i]), df_mask['Quantity'][i], df_mask['Unit Value ($)'][i])
        trans_list.append(trans)
    trans_result = balanceFifo(trans_list)
    df_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])

    # Filtering sold date for 2021-2022 Capital Gain
    df_capital['Date Sold'] = pd.to_datetime(df_capital['Date Sold'])
    mask_time = (df_capital['Date Sold'] > start_date) & (df_capital['Date Sold'] < end_date)
    df_filtered = df_capital[mask_time]

    print("Financial Year Capital gain for", code.index[j])
    print(df_filtered)
    
    # Summation of capita gains (Total: any loses, discounts not included, Net: including loses or discounts)
    sum_total = np.sum(df_filtered['Capital Gain'].where(df_filtered['Capital Gain'] > 0, 0))
    sum_net = np.sum(df_filtered['Capital Gain'])
    
    #For net capital gain, discounts for 'over 1 year stocks' should be applied
    #diff = df_filtered['Date Sold'] - df_filtered['Date Purchased'] # How many days holding stocks
    #discount = 0 # Amount of discount
    df_filtered['Days Held'] = (df_filtered['Date Sold'] - df_filtered['Date Purchased']).dt.days
    discount = np.sum(df_filtered.loc[df_filtered['Days Held'] > 365, 'Capital Gain'] / 2)
    sum_net -= discount
    
    # If you hold stocks more than a year, you get half of discount on capital gain for taxation
    #for y in range(0, diff.size):
    #    if (int(diff.values[y]/8.64e+13) > 365): 
    #        discount = discount + df_filtered['Capital Gain'][y] / 2
    #        sum_net = sum_net - discount
    #    else: 
    #        sum_net = sum_net
        
    print("Total capital gain for", code.index[j], ": ", round(sum_total, 2), "AUD")
    print("Net capital gain for", code.index[j], ": ", round(sum_net, 2), "AUD")
    
    # Append data into Summary dataframe
    a = [code.index[j], sum_total, sum_net]
    sum_df.append(a)

All Transactions for SQQQ
    Code       Date Type  Quantity  Total Value ($)  Unit Value ($)
0   SQQQ 2022-07-19    B         2       143.301744       71.650872
1   SQQQ 2022-07-20    B         2       138.921120       69.460560
2   SQQQ 2022-07-20    B         2       138.950080       69.475040
3   SQQQ 2022-07-20    S        -6      -412.578640       68.763107
4   SQQQ 2022-07-21    B         5       332.670416       66.534083
..   ...        ...  ...       ...              ...             ...
59  SQQQ 2023-02-23    S        -5      -279.810978       55.962196
60  SQQQ 2023-02-23    S        -5      -283.089198       56.617840
61  SQQQ 2023-03-15    B        60      3326.887674       55.448128
62  SQQQ 2023-04-03    B       100      4468.423310       44.684233
63  SQQQ 2023-06-09    B        38      1220.849140       32.127609

[64 rows x 6 columns]
Financial Year Capital gain for SQQQ
   Date Purchased  Date Sold  Quantity  Buy Price  Sell Price  Capital Gain
0      2022-07-19 2022

/var/folders/s3/9552g_5j4bg7zm2wcp_ldt4h0000gn/T/ipykernel_3590/4275349133.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Days Held'] = (df_filtered['Date Sold'] - df_filtered['Date Purchased']).dt.days
/var/folders/s3/9552g_5j4bg7zm2wcp_ldt4h0000gn/T/ipykernel_3590/4275349133.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Days Held'] = (df_filtered['Date Sold'] - df_filtered['Date Purchased']).dt.days


In [6]:
# Print summary dataframe for total and net capital gain
summary = pd.DataFrame(sum_df, columns = ['Code', 'Total gain', 'Net gain'])
print(summary)
print("Total Capital Gain: ", summary['Total gain'].sum().round(), "AUD")
print("Net Capital Gain: ", summary['Net gain'].sum().round(), "AUD")

    Code   Total gain     Net gain
0   SQQQ  1238.155216   519.541121
1   TQQQ   899.409720   509.804668
2   SOXL  1981.097776  1933.128256
3   AAPL     0.000000  -473.835522
4   QYLD     1.919028     0.847740
5   TSLA     0.000000  -224.444204
6    CVX    56.766052    56.766052
7    OXY    65.537960    65.537960
8    DBA     0.000000     0.000000
9    JNJ     2.934451     2.934451
10  SNOW     0.000000   -16.094520
Total Capital Gain:  4246.0 AUD
Net Capital Gain:  2374.0 AUD


In [7]:
summary.to_excel('tax_return_2023_US.xlsx', sheet_name = 'US_CG')